# Домашнее задание № 6, Кривоногов Н.В.

Код для recommenders, utils, metrics скачал из [этого](https://github.com/geangohn/recsys-tutorial) github репозитория

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from metrics import precision_at_k, recall_at_k
from utils import prefilter_items
from recommenders import MainRecommender

In [3]:
data = pd.read_csv('retail_train.csv')
item_features = pd.read_csv('product.csv')
user_features = pd.read_csv('hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)


# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
n_items_before = data_train_lvl_1['item_id'].nunique()

data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features=item_features, take_n_popular=5000)

n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 83685 to 5001


In [5]:
data_train_lvl_1['user_id'].unique()

array([2375, 1364, 1172, ...,  363, 1721, 1480], dtype=int64)

In [6]:
recommender = MainRecommender(data_train_lvl_1)

C:\Conda\lib\site-packages\implicit\utils.py:33: UserWarning: Intel MKL BLAS detected. Its highly recommend to set the environment variable 'export MKL_NUM_THREADS=1' to disable its internal multithreading
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

### Задание 1

A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?
- Пока пробуем отобрать 50 кандидатов (k=50)
- Качество измеряем на data_val_lvl_1: следующие 6 недель после трейна

Дают ли own recommendtions + top-popular лучший recall?  

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}  
C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?


In [7]:
train_users = data_train_lvl_1['user_id'].unique() # warm start

result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1 = result_lvl_1.loc[result_lvl_1.user_id.isin(train_users)]
result_lvl_1.columns=['user_id', 'actual']
result_lvl_1.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


Часть А:

In [8]:
result_lvl_1['own_candidates'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_own_recommendations(x, 200))

In [9]:
result_lvl_1['own_recall'] = result_lvl_1.apply(lambda row: recall_at_k(row['own_candidates'], row['actual'], k=200), axis=1)

In [10]:
result_lvl_1['own_recall'].mean()

0.08921574430574228

In [11]:
result_lvl_1['als_candidates'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_als_recommendations(x, 200))

In [12]:
result_lvl_1['als_recall'] = result_lvl_1.apply(lambda row: recall_at_k(row['als_candidates'], row['actual'], k=200), axis=1)

In [13]:
result_lvl_1['als_recall'].mean()

0.08921574430574228

In [14]:
result_lvl_1['similar_item_candidates'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_similar_items_recommendation(x, 200))

In [15]:
result_lvl_1['similar_item_recall'] = result_lvl_1.apply(lambda row: recall_at_k(row['similar_item_candidates'], row['actual'], k=200), axis=1)

In [16]:
result_lvl_1['similar_item_recall'].mean()

0.08943004283810192

Лучший результат дают similar_items_recommendation. 

### Задание 2.

Обучите модель 2-ого уровня, при этом:
    - Добавьте минимум по 2 фичи для юзера, товара и пары юзер-товар
    - Измерьте отдельно precision@5 модели 1-ого уровня и двухуровневой модели на data_val_lvl_2
    - Вырос ли precision@5 при использовании двухуровневой модели?

In [17]:
result_lvl_1['als_precision'] = result_lvl_1.apply(lambda row: precision_at_k(row['als_candidates'], row['actual'], k=200), axis=1)

In [18]:
als_candidates = result_lvl_1[['user_id', 'als_candidates']]
als_candidates.head()

,user_id,als_candidates
0,1,"[999999, 1029743, 1106523, 5569230, 916122, 84..."
1,2,"[999999, 1029743, 1106523, 5569230, 916122, 84..."
2,4,"[999999, 1029743, 1106523, 5569230, 916122, 84..."
3,6,"[999999, 1029743, 1106523, 5569230, 916122, 84..."
4,7,"[999999, 1029743, 1106523, 5569230, 916122, 84..."


In [19]:
valid_lvl_2 = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index().rename(columns={'item_id': 'actual'})

In [20]:
valid_lvl_2 = valid_lvl_2.merge(als_candidates, on='user_id', how='left')

Точность ALS: 

In [21]:
valid_lvl_2[valid_lvl_2.als_candidates.notna()].apply(lambda row: precision_at_k(row['als_candidates'], row['actual'], k=5), axis=1).mean()

0.104543080939947

In [22]:
users_lvl_2 = pd.DataFrame(result_lvl_1['user_id'].unique())
users_lvl_2.columns = ['user_id']

train_users = data_train_lvl_1['user_id'].unique()
users_lvl_2 = users_lvl_2[users_lvl_2['user_id'].isin(train_users)]

users_lvl_2['candidates'] = users_lvl_2['user_id'].apply(lambda x: recommender.get_als_recommendations(x, 200))

In [23]:
s = users_lvl_2.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'item_id'

In [24]:
users_lvl_2 = users_lvl_2.drop('candidates', axis=1).join(s)
users_lvl_2['flag'] = 1

users_lvl_2.tail(4)

,user_id,item_id,flag
2150,2500,965842,1
2150,2500,825994,1
2150,2500,12262778,1
2150,2500,854754,1


In [25]:
data_train_lvl_2 = data_val_lvl_1.copy()

In [26]:
targets_lvl_2 = data_train_lvl_2[['user_id', 'item_id']].copy()
targets_lvl_2['target'] = 1  

targets_lvl_2 = users_lvl_2.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')

targets_lvl_2['target'].fillna(0, inplace= True)
targets_lvl_2.drop('flag', axis=1, inplace=True)

Добавляю фичи: 

In [27]:
price = data.groupby('item_id')['sales_value'].max()

In [28]:
item_features = item_features.merge(price, on='item_id', how='left').rename(columns={'sales_value': 'price'})
item_features.tail(5)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,price
92348,18293142,6384,DRUG GM,National,BOOKSTORE,PAPERBACK BOOKS,,NaN
92349,18293439,6393,DRUG GM,National,BOOKSTORE,CHILDRENS LOW END,,NaN
92350,18293696,6406,DRUG GM,National,BOOKSTORE,PAPERBACK BEST SELLER,,NaN
92351,18294080,6442,DRUG GM,National,BOOKSTORE,PAPERBACK BOOKS,,NaN
92352,18316298,764,GROCERY,National,PAPER TOWELS,PAPER TOWELS & HOLDERS,,NaN


Средняя цена по категории: 

In [29]:
avg_price_by_cat = item_features.groupby('commodity_desc')['price'].mean()
item_features = item_features.merge(avg_price_by_cat, on='commodity_desc', how='left').rename(columns={'price_x': 'price', 'price_y': 'avg_price'})

In [30]:
item_features.head()

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,price,avg_price
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB,13.96,6.811379
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,,0.99,6.879213
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,,1.59,3.121493
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ,1.54,4.182812
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ,1.98,3.730524


Суммарная выручка с пользователя и средняя выручка за неделю: 

In [31]:
basket_price = data.groupby('basket_id')['sales_value'].sum()

In [32]:
data = data.merge(basket_price, on='basket_id', how='left').rename(columns={'sales_value_x': 'sales_value', 'sales_value_y': 'basket_price'})

In [33]:
avg_basket_per_user = data.groupby('user_id').basket_price.mean()

In [34]:
user_features = user_features.merge(avg_basket_per_user, on='user_id', how='left')

In [35]:
x = data.groupby(['user_id', 'week_no']).basket_id.count().reset_index().groupby('user_id').agg(['count', 'sum'])

In [36]:
x.columns = x.columns.droplevel(0)

In [37]:
avg_purchases_per_week = x.iloc[:, 2:].apply(lambda row: row['sum']/row['count'], axis=1)
avg_purchases_per_week.name = 'avg_purchases_per_week'

In [38]:
user_features = user_features.merge(avg_purchases_per_week, on='user_id', how='left')

In [39]:
user_features.head()

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id,basket_price,avg_purchases_per_week
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1,77.419817,24.828125
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7,86.418470,24.363636
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8,82.333861,26.409091
3,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2,13,91.693154,26.750000
4,45-54,B,50-74K,Homeowner,Single Female,1,None/Unknown,16,21.855595,9.160714


In [40]:
item_features_copy = item_features[['item_id',
                                    'manufacturer',
                                    'department',
                                    'brand',
                                    'commodity_desc',
                                    'price',
                                    'avg_price']]

user_features_copy = user_features[['user_id',
                                    'age_desc',
                                    'income_desc',
                                    'household_size_desc',
                                    'avg_purchases_per_week',
                                    'basket_price']]

In [41]:
targets_lvl_2 = targets_lvl_2.merge(item_features_copy, on='item_id', how='left')
targets_lvl_2 = targets_lvl_2.merge(user_features_copy, on='user_id', how='left')

targets_lvl_2.head()

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,price,avg_price,age_desc,income_desc,household_size_desc,avg_purchases_per_week,basket_price
0,1,999999,0.0,NaN,NaN,NaN,NaN,NaN,NaN,65+,35-49K,2,24.828125,77.419817
1,1,1029743,0.0,69.0,GROCERY,Private,FLUID MILK PRODUCTS,15.04,4.110450,65+,35-49K,2,24.828125,77.419817
2,1,1106523,0.0,69.0,GROCERY,Private,FLUID MILK PRODUCTS,29.88,4.110450,65+,35-49K,2,24.828125,77.419817
3,1,5569230,0.0,1208.0,GROCERY,National,SOFT DRINKS,91.61,5.947091,65+,35-49K,2,24.828125,77.419817
4,1,916122,0.0,4314.0,MEAT,National,CHICKEN,92.94,9.049487,65+,35-49K,2,24.828125,77.419817


In [42]:
X_train = targets_lvl_2.drop('target', axis=1)
y_train = targets_lvl_2['target']

In [43]:
cat = ['manufacturer',
               'department',
               'brand',
               'commodity_desc',
               'age_desc',
               'income_desc',
               'household_size_desc']

In [44]:
X_train[cat] = X_train[cat].astype('category')

In [45]:
lgb = LGBMClassifier(objective='binary', max_depth=7)
lgb.fit(X_train, y_train.values)

train_preds = lgb.predict_proba(X_train)[:, 1]

In [46]:
targets_lvl_2['preds'] = train_preds

In [47]:
targets_lvl_2.sort_values(['user_id', 'preds'], ascending=[True, False], inplace=True)

In [48]:
lgb_candidates = targets_lvl_2.groupby('user_id').head(5).groupby('user_id')['item_id'].unique().reset_index()

In [49]:
lgb_candidates.head()

,user_id,item_id
0,1,"[1082185, 1029743, 1106523]"
1,2,"[1082185, 1029743, 1106523, 1126899, 1068719]"
2,4,"[1082185, 1029743, 1106523, 1126899, 1068719]"
3,6,"[1082185, 1029743, 1106523, 1126899]"
4,7,"[1082185, 1029743, 1106523, 1068719]"


In [50]:
valid_lvl_2 = valid_lvl_2.merge(lgb_candidates, on='user_id', how='left')
valid_lvl_2.rename(columns={'item_id': 'lgb_candidates'}, inplace=True)

In [51]:
valid_lvl_2[valid_lvl_2.als_candidates.notna()].apply(lambda row: precision_at_k(row['lgb_candidates'], row['actual'], k=5), axis=1).mean()

0.24447345517841418

Метрика precision@k выросла в 2.33 раза. 